In [1]:
import pandas as pd
import os
import torch
from models_utils.GLOBALS import files_directory
from RF_XGB.RandomForest import get_rf_data
from RF_XGB.XGBoost import train_xgb
from models_utils.utils import convert_to_features

In [2]:
# get train data
data_type_1, data_type_2 = get_rf_data()

In [4]:
# train xgb
xgb_classifier_type1, label_encoder_type1 = train_xgb(data_type_1, ['activity'], 200)
xgb_classifier_type2, label_encoder_type2 = train_xgb(data_type_2, ['activity'], 200)

Accuracy: 0.829234595192042
Log Loss: 0.6229548565438214
Accuracy: 0.7621756132243157
Log Loss: 0.7809197322508317


In [ ]:
# get and save results for test data
labels = ['brushing_teeth', 'idle', 'preparing_sandwich', 'reading_book', 'stairs_down', 'stairs_up', 'typing',
          'using_phone', 'using_remote_control', 'walking_freely', 'walking_holding_a_tray',
          'walking_with_handbag', 'walking_with_hands_in_pockets', 'walking_with_object_underarm',
          'washing_face_and_hands', 'washing_mug', 'washing_plate', 'writing']
results_list = []

for i, file_id in enumerate(pd.read_csv('sample_submission.csv')['sample_id'].to_list()):

    class_path = os.path.join(files_directory, f"{file_id}.csv")
    new_data = pd.read_csv(class_path)
    if new_data.shape[1] == 4:
        new_data = new_data[new_data.iloc[:, 0] == 'acceleration [m/s/s]'].iloc[:, 1:]
        data_x_tensor = torch.tensor(new_data["x"].values, dtype=torch.float32)
        data_y_tensor = torch.tensor(new_data["y"].values, dtype=torch.float32)
        data_z_tensor = torch.tensor(new_data["z"].values, dtype=torch.float32)
        res = xgb_classifier_type1.predict_proba(convert_to_features(data_x_tensor, data_y_tensor, data_z_tensor))
        res_dict = dict(zip(label_encoder_type1.inverse_transform(xgb_classifier_type1.classes_), res[0]))
    else:
        data_x_tensor = torch.tensor(new_data["x [m]"].values, dtype=torch.float32)
        data_y_tensor = torch.tensor(new_data["y [m]"].values, dtype=torch.float32)
        data_z_tensor = torch.tensor(new_data["z [m]"].values, dtype=torch.float32)
        res = xgb_classifier_type2.predict_proba(convert_to_features(data_x_tensor, data_y_tensor, data_z_tensor))
        res_dict = dict(zip(label_encoder_type2.inverse_transform(xgb_classifier_type2.classes_), res[0]))

    result_dict = {label: res_dict.get(label, 0) for label in labels}
    result_dict['sample_id'] = file_id
    results_list.append(result_dict)

results = pd.DataFrame(results_list, columns=['sample_id'] + labels)
results.to_csv('XGB_res3.csv', index=False)